In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import bs4
import requests

import json
import re
from tqdm import notebook

In [5]:
# json 파일 생성
data = {}
data["data"] = []

with open('recipe_data.json', 'w', encoding="utf-8") as make_file:    
    json.dump(data, make_file, ensure_ascii=False, indent="\t")

In [6]:
header = {"User-Agent" : "Mozilla/5.0"}

# 해당 사이트에 총 25개의 카테고리가 있음을 확인(1 ~ 25)
for i in notebook.tqdm(range(25)): # 0 ~ 24
    theme_url = "https://2bob.co.kr/recipe.php?id=list&fKeyList=&fKeyValue=&eTheme={}"
    theme_url = theme_url.format(i+1)


    # 한개의카테고리에 몇개의 큰 페이지(5개 묶음 ex> 1 ~ 5, 6 ~ 10, 11 ~ 12)가 있는지 파악
    # ex) 3페이지일 경우 리스트가 [1, 6, 11]
    # ex) 2페이지일 경우 리스트가 [1. 6]
    next_page_num_list = [1]
    k = 0
    while(True):
        # 웹 소스 준비
        url = theme_url + "&OrderCondition=&OrderBy=&page={}"      
        url = url.format(next_page_num_list[k])
        page = requests.get(url, headers=header)

        soup = BeautifulSoup(page.text, "html.parser")

        if(soup.find("img", alt="마지막으로") != None):    
            # 페이지에서 >> 버튼("마지막으로")이 있을경우 다음 큰 페이지가 있음을 의미함
            next_Page = soup.find("img", alt="마지막으로").parent
        else:
            break   # 마지막으로 버튼이 없을 경우 해당 카테고리의 마지막 큰 페이지이므로 반복문 종료
        if(next_page_num_list[k] < int(next_Page["href"].split("=")[-1])):
            next_page_num_list.append(int(next_Page["href"].split("=")[-1]))
            k = k + 1
#             print(next_Page["href"].split("=")[-1])
        else:
            break

#     print(next_page_num_list)

    for lead_page_num in notebook.tqdm(next_page_num_list):

        # 웹 소스 준비
        url = theme_url + "&OrderCondition=&OrderBy=&page={}"     
        url = url.format(lead_page_num)

        
        page = requests.get(url, headers=header)

        # 가져온 웹 소스를 트리 구조로 변환
        soup = BeautifulSoup(page.text, "html.parser")

        # urljoin을 위한 base 선언
        base = "https://2bob.co.kr"
        base2 = "https://2bob.co.kr/recipe.php"

        # 한개의 카테고리의 페이지 url 가져오기
        page_list = []
        page_list.append(url)
        page_list.append(soup.find("p", "paging_num").a["href"])
        next_pages = soup.find("p", "paging_num").a.next_siblings

        for page in next_pages:
            page_list.append(page["href"])

        for page in notebook.tqdm(page_list):
            if(page == url):
                url = page
            else:
                url = base2 + page

            page = requests.get(url, headers=header)

            # 가져온 웹 소스를 트리 구조로 변환
            soup = BeautifulSoup(page.text, "html.parser")

            # 페이지당 4줄, 4줄의 맨 앞에있는 레시피만 ml_none 클래스 부여되어있음
            # 따라서 뒤에 따라오는 4개의 레시피는 해당 레시피 기준 siblings을 사용하여 가져옴

            # 가로기준 맨 앞 항목의 링크 가져오기
            urls = soup.select('li.ml_none > a')
            lead_urls_list = []
            for url in urls:
                lead_urls_list.append(url["href"])

            # 맨 앞 항목의 링크를 기준으로 따라오는 4개의 링크 가져오기
            follow_url_list=[]
            for lead_url in lead_urls_list:
                urls = soup.find("a", href=lead_url).parent.next_siblings
                for url in urls:
                    if(type(url) == bs4.element.Tag):
                        follow_url_list.append(list(url.children)[1]["href"])

            # 두개의 리스트 합치기
            url_list = lead_urls_list + follow_url_list

            # 합쳐진 리스트의 중복 제거
            url_list = set(url_list)
            url_list = list(url_list)

            # 괄호, \r\n 제거를 위한 패턴 생성
            pattern1 = r'\([^)]*\)'
            pattern2 = '\r\n'
            pattern3 = '\n'

            # 생성해 놓은 json파일 load
            with open('recipe_data.json', "r", encoding='utf-8') as json_file:    
                json_data = json.load(json_file)

                    
            if(len(json_data["data"]) == 0): # 처음 저장할 경우
                cnt = 0
            else: # 기존에 테이터가 있을 경우
                cnt = json_data["data"][-1]["code"] + 1   

            for i in notebook.tqdm(url_list):
                # 웹 소스 준비
                url = urllib.parse.urljoin(base, i)

                header = {"User-Agent" : "Mozilla/5.0"}
                page = requests.get(url, headers=header)
                page
                # 가져온 웹 소스를 트리 구조로 변환
                soup = BeautifulSoup(page.text, "html.parser")

                # 레시피 명
                title = soup.find('h2', class_="prod_title").get_text()
#                 print(title)

                # 필수재료
                ingredients = soup.find('p', class_="mate_list").get_text()

                # 재료명에서 괄호와 괄호안의 정보 삭제
                ingredients = re.sub(pattern=pattern1, repl='', string=ingredients)

                # 재료명에서 \r\n 제거
                ingredients = re.sub(pattern=pattern2, repl='', string=ingredients)
                
                # 재료명에서 \n 제거
                ingredients = re.sub(pattern=pattern3, repl='', string=ingredients)

                # 리스트로 변환
                temp_list = ingredients.split(',')

                ingredients_list = []
                # 공백 제거
                for i in temp_list:
                    i = re.sub(" ", "", string=i)
                    ingredients_list.append(i)

#                 print(ingredients_list)


                # 필수재료 이외의 재료와 양념이 있다면 가져오기
                if(soup.find('p', class_="mate_list").parent.next_siblings != None):
                    siblings = soup.find('p', class_="mate_list").parent.next_siblings
                    for sibling in siblings:
                        if(type(sibling) == bs4.element.Tag):

                            if(sibling.p != None):

                                ingredients = sibling.p.get_text()

                                # 재료명에서 괄호와 괄호안의 정보 삭제
                                ingredients = re.sub(pattern=pattern1, repl='', string=ingredients)

                                # 재료명에서 \r\n 제거
                                ingredients = re.sub(pattern=pattern2, repl='', string=ingredients)

                                # 재료명에서 \n 제거
                                ingredients = re.sub(pattern=pattern3, repl='', string=ingredients)

                                # 리스트로 변환
                                temp_list = ingredients.split(',')


                                # 공백 제거
                                for i in temp_list:
                                    i = re.sub(" ", "", string=i)
                                    ingredients_list.append(i)
                
                # 특수 양념 예외처리
                for ingredient in ingredients_list:
                    if("+" in ingredient):
                        temp = ingredient.split("+")
                        ingredients_list.remove(ingredient)
                        ingredients_list = ingredients_list + temp
    
                json_data['data'].append({
                    "code" : cnt,
                    "title" : title,
                    "ingredients" : ingredients_list,
                    "url" : url
                })
#                 print("--------------------------------------------------")
                cnt = cnt + 1
                # json 파일 저장
                with open('recipe_data.json', "w", encoding='utf-8') as outfile:      ####################################### 수정해야할 곳 ####################
                    json.dump(json_data, outfile, indent="\t", ensure_ascii=False)


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]